# Python Dictionaries, A very clever data structure!

In [1]:
talk = {
    "title": "Python Dictionaries, A very clever data structure!",
    "author": "Kai Striega"
}

# What is a dictionary (Revision)

A dictionary is a mapping of ``(key, value)`` pairs. Unlike sequences, which are index by integers, dictionaries are indexed by _keys_. The key can be any hashable Python object. 


## Importance

Dictionaries play an extremely important role in Python. Not only are dictionaries used in almost every python program, the Python interpreter also requires dictionaries to _run_ the Python code.

## Operations

There are many operations that can be performed on dictionaries. The full list is available in the [docs](https://docs.python.org/3/library/stdtypes.html#dict). For the sake of brevity our dictionary will be defined as having three main operations:

* Insert new items ``d[k] = v``
* Lookup the value of a give key ``d[k]``
* Delete a ``(key, value)`` pair from the dictionary ``del d[k]``

We can verify with our example dictionary from above that we can do all these operations

In [2]:
# ``d[k] = v`` adds the ``(k, v)`` pair to the dictionary
talk["audience"] = "SheCodes"
# We can see our dict now contains an ``audience`` key and ``SheCodes`` value
talk  

{'title': 'Python Dictionaries, A very clever data structure!',
 'author': 'Kai Striega',
 'audience': 'SheCodes'}

In [3]:
# ``d[k]`` returns the value assoicated with ``k``
talk["author"]

'Kai Striega'

In [4]:
# ``del d[k]`` delets the value associated with ``k``
del talk["title"]
talk

{'author': 'Kai Striega', 'audience': 'SheCodes'}

# Implementation

## Implementation vs Interface

Dictionaries are an _interface_. It defines how an object should behave given different operations. There are several ways to _implement_ this interface:

* A [Hashmap/Hash table](https://en.wikipedia.org/wiki/Hash_table)
* A [Linked List](https://en.wikipedia.org/wiki/Linked_list) with linear search for the elements
* A [Search Tree](https://en.wikipedia.org/wiki/Search_tree)

Python chooses to implement using a hashmap. This allows for very perfomant _average case_ operations. This talk covers how to implement a dictionary using a hashmap, and the optimisations that Python uses to increase their performance.

## Objective

A computer doesn't understand complex data structures such as dictionaries, lists or tuples. All of these complex data structures had to be written by someone. In the rest of this talk we will implement a hashmap from scratch. Using only what the computer has given us. CPython does this for dictionaries in about [5000 lines of C](https://github.com/python/cpython/blob/3.12/Objects/dictobject.c), however we'll be working in Python using some abstractions.

## And first, there was memory

To begin our hashmap we will need to get some memory. As we haven't put anything into it yet, let's represent it as a list of ``None`` objects. For now I've chosen to store 8 ``None`` objects. This is a very small amount of memory. Don't worry - we'll figure out how to add more items later.

In [5]:
# Mimick 8 "slots" in memory
memory = [None] * 8
memory

[None, None, None, None, None, None, None, None]

## Hashing

Now we have some memory to work with, and we can access our memory using an index.  But that isn't what we want to do. We want to be able to use some key to access the underlying value. To do this we will use what's called a [Hash Function](https://en.wikipedia.org/wiki/Hash_function).

A hash function takes arbitrarily large data and converts it to fixed size data. We can then interperate that bit pattern as an integer and use that integer as an index. Designing good hash functions is difficult (more on that later), luckily Python has an inbuilt hash function - [hash](https://docs.python.org/3/library/functions.html#hash). Let's play around with hash for a bit.

In [6]:
hash(10)

10

In [7]:
hash("SheCodes")

6051908547441681576

In [8]:
hash((10, "SheCodes"))

-1218130345949401146

These integers are far too large to fit into our small hashtable. But we can take the modulus of the integer to get our index. This way we will always find a way to store the given key.

In [9]:
hash("SheCodes") % 8

0

In a hashmap, each entry is refered to as a bucket. Let's create a quick class to store each bucket in our memory

In [10]:
from dataclasses import dataclass
from typing import Hashable, Any

@dataclass
class Bucket:
    key: Hashable
    value: Any

Now we may assign a ``(key, value)`` pair by creating a bucket and assigning it to the memory address we calculated using the hash

In [11]:
key = "Kai"
value = "is awesome"

h = hash(key)
i = h % 8
memory[i] = Bucket(key, value)

In [12]:
memory

[Bucket(key='Kai', value='is awesome'),
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [13]:
memory[i]

Bucket(key='Kai', value='is awesome')

## Wrapping it all into a class

We've done quite a bit of work here. We've found a way to mimick allocating memory, assing objects to our "dictionary" and to access objects. Let's wrap it all in a class so that we can reuse our dictionary more easily 

In [14]:
from typing import Optional


class Dictionary:

    def __init__(self, init_size=8):
        self._size = init_size
        self._data: list[Optional[Bucket]] = [None] * self._size

    def __setitem__(self, key, value):
        h = hash(key)
        i = h % self._size
        bucket = Bucket(key, value)
        self._data[i] = bucket
        
    def __getitem__(self, item):
        h = hash(item)
        i = h % self._size
        return self._data[i].value 

That's better. Let's experiment with our new class a bit!

In [15]:
d = Dictionary()
d[0] = "Kai"
d[1] = "is awesome"
d._data

[Bucket(key=0, value='Kai'),
 Bucket(key=1, value='is awesome'),
 None,
 None,
 None,
 None,
 None,
 None]

## Collisions

In [16]:
d[8] = "SheCodes"
d._data

[Bucket(key=8, value='SheCodes'),
 Bucket(key=1, value='is awesome'),
 None,
 None,
 None,
 None,
 None,
 None]

What's happened here?

Because ``hash(0) % 8 == hash(8) % 8`` the index is the same, so we've overwritten the existing data in our dictionary. This is called a [collision](https://en.wikipedia.org/wiki/Hash_table#Collision_resolution). There are two common ways of resolving collisions: Separate Chaining and Open Addressing

### Separate Chaining

![chaining](static/images/hash_table_with_chaining.png)

### Open Addressing

The problem with chaining is that we're allocating more memory than we need to. What if, instead of allocating new memory, we use the memory we have already assigned? It's possible to do this by storing the ``(key, value)`` pairs in exisitng buckets, then instead of generating an index using our hash we generate a _sequence_ of indecies. This process is called _probing_ and the sequence we generate is called a _probe sequence_. To add a new item to our dictionary we check each index generated by the probe sequence and assign it to the first empty slot. There are three common types of prove sequences, which we'll explore now!

#### Linear Probing

$$ probes[i] = hash(key) + i \space \% \space number \space of \space buckets$$

#### Quadratic Probing

$$ probes[i] = hash(key) + a * i + b * i^2 \space \% \space number \space of \space buckets $$ 

#### Pseudo-Random Probing

$$ probes[i] = a * probes[i-1] + c \space \% \space number \space of \space buckets $$

In [17]:
def probes(hash_value, hash_table_size):
    mask = hash_table_size - 1 # used to take modulus fast
    perturb = hash_value # used to perturb the probe sequence
    probe = hash_value & mask

    while True:
        yield probe

        perturb >>= 5
        probe = (probe * 5 + perturb + 1) & mask

In [18]:
class Dictionary:

    def __init__(self, init_size=8):
        self._size = init_size
        self._data: list[Optional[Bucket]] = [None] * self._size
                
    def _probes(self, hash_value):
        mask = self._size - 1 # used to take modulus fast
        perturb = hash_value # used to perturb the probe sequence
        probe = hash_value & mask
    
        while True:
            yield probe
    
            perturb >>= 5
            probe = (probe * 5 + perturb + 1) & mask
        
        
    def __setitem__(self, key, value):
        h = hash(key)
        bucket = Bucket(key, value)
        
        for probe in self._probes(h):
            if self._data[probe] is None:
                self._data[probe] = bucket
                break
            elif self._data[probe].key == bucket.key:
                self._data[probe] = bucket
                break
        
    def __getitem__(self, item):
        h = hash(item)
        for probe in self._probes(h):
            if self._data[probe] is None:
                raise KeyError(item)
            elif self._data[probe].key == item:
                return self._data[probe].value
    

In [19]:
d = Dictionary()
d[0] = "Kai"
d[1] = "is awesome"

In [20]:
d._data

[Bucket(key=0, value='Kai'),
 Bucket(key=1, value='is awesome'),
 None,
 None,
 None,
 None,
 None,
 None]

In [21]:
d[8] = "SheCodes"
d._data

[Bucket(key=0, value='Kai'),
 Bucket(key=1, value='is awesome'),
 None,
 None,
 None,
 None,
 Bucket(key=8, value='SheCodes'),
 None]

In [22]:
d[0] = "Not Kai"
d._data

[Bucket(key=0, value='Not Kai'),
 Bucket(key=1, value='is awesome'),
 None,
 None,
 None,
 None,
 Bucket(key=8, value='SheCodes'),
 None]

## Reshaping

So far we've been dealing with a fixed size of 8 buckets in a dictionary. This works well but what if we want to store more than 8 buckets? One solution would be to create a bigger, fixed size, dictionary. But Python uses many small dictionaries, so increasing the size would lead to a lot of empty (wasted) buckets. What we're going to do instead is to dynamically reshape our buckets.

### Load Factor

### Choosing a sizing scheme

#### Primes

#### Powers of 2




In [23]:
class Dictionary:

    def __init__(self, init_size=8):
        self._items = 0
        self._size = init_size
        self._data: list[Optional[Bucket]] = [None] * self._size
        self._reshape_threshold = 2/3

    @property
    def load_factor(self):
        return self._items / self._size
        
        
    def _probes(self, hash_value):
        mask = self._size - 1 # used to take modulus fast
        perturb = hash_value # used to perturb the probe sequence
        probe = hash_value & mask
    
        while True:
            yield probe
    
            perturb >>= 5
            probe = (probe * 5 + perturb + 1) & mask
        

    def _reshape(self):
        self._size <<= 1
        new_buckets = [None] * self._size
        for bucket in self._data:
            if bucket: # Avoid copying empty buckets
                h = hash(bucket.key)
                for probe in self._probes(h):
                    if new_buckets[probe] is None:
                        new_buckets[probe] = bucket
                        break
        self._data = new_buckets
        
    def __setitem__(self, key, value):
        h = hash(key)
        bucket = Bucket(key, value)
        
        for probe in self._probes(h):
            if self._data[probe] is None:
                self._data[probe] = bucket
                self._items += 1
                break
            elif self._data[probe].key == bucket.key:
                self._data[probe] = bucket
                break

        if self.load_factor > self._reshape_threshold:
            self._reshape()
            
    
    def __getitem__(self, item):
        h = hash(item)
        for probe in self._probes(h):
            if self._data[probe] is None:
                raise KeyError(item)
            elif self._data[probe].key == item:
                return self._data[probe].value
    

In [24]:
d = Dictionary()
for i in range(5):
    d[i] = i
d._data

[Bucket(key=0, value=0),
 Bucket(key=1, value=1),
 Bucket(key=2, value=2),
 Bucket(key=3, value=3),
 Bucket(key=4, value=4),
 None,
 None,
 None]

In [25]:
# We can update values without resizing
d[0] = 15
d._data

[Bucket(key=0, value=15),
 Bucket(key=1, value=1),
 Bucket(key=2, value=2),
 Bucket(key=3, value=3),
 Bucket(key=4, value=4),
 None,
 None,
 None]

In [26]:
# We resize if another element is added
d[5] = 5
d._data

[Bucket(key=0, value=15),
 Bucket(key=1, value=1),
 Bucket(key=2, value=2),
 Bucket(key=3, value=3),
 Bucket(key=4, value=4),
 Bucket(key=5, value=5),
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

## Deleting Items

We've covered two of our three use cases extensively, but what if we want to delete items from a dictionary? This is more complicated with open addressing as removing the bucket would disrupt our probe sequence. What we're going to do instead is to mark objects as deleted, then have another step to remove the deleted items. Let's update our ``Bucket`` class to reflect this.

In [27]:
@dataclass
class Bucket:
    key: Hashable
    value: Any
    is_valid: bool = True

We also have to update our dictionary class to check if a bucket is deleted before returning it:

In [28]:
class Dictionary:

    def __init__(self, init_size=8):
        self._items = 0
        self._size = init_size
        self._data: list[Optional[Bucket]] = [None] * self._size
        self._reshape_threshold = 2/3

    @property
    def load_factor(self):
        return self._items / self._size
        
        
    def _probes(self, hash_value):
        mask = self._size - 1 # used to take modulus fast
        perturb = hash_value # used to perturb the probe sequence
        probe = hash_value & mask
    
        while True:
            yield probe
    
            perturb >>= 5
            probe = (probe * 5 + perturb + 1) & mask
        

    def _reshape(self):
        self._size <<= 1
        new_buckets = [None] * self._size
        for bucket in self._data:
            if bucket and bucket.is_valid:
                h = hash(bucket.key)
                for probe in self._probes(h):
                    if new_buckets[probe] is None:
                        new_buckets[probe] = bucket
                        break
        self._data = new_buckets
        
    def __setitem__(self, key, value):
        h = hash(key)
        new_bucket = Bucket(key, value)
        
        for probe in self._probes(h):
            bucket = self._data[probe]
            if bucket is None or not bucket.is_valid:
                self._data[probe] = new_bucket
                self._items += 1
                break
            elif bucket.key == new_bucket.key:
                self._data[probe] = new_bucket
                break

        if self.load_factor > self._reshape_threshold:
            self._reshape()
            
    
    def __getitem__(self, item):
        h = hash(item)
        for probe in self._probes(h):
            bucket = self._data[probe]
            if bucket is None:
                raise KeyError(item)
            elif bucket and bucket.key == item:
                return self._data[probe].value

    def __delitem__(self, key):
        h = hash(key)
        for probe in self._probes(h):
            bucket = self._data[probe]
            if bucket is not None and bucket.is_valid and bucket.key == key:
                bucket.is_valid = False
                self._items -= 1
                return
        raise KeyError(key)

In [29]:
Bucket(5, 15)

Bucket(key=5, value=15, is_valid=True)

In [30]:
d = Dictionary()

In [31]:
d[1] = 1

In [32]:
d._data

[None,
 Bucket(key=1, value=1, is_valid=True),
 None,
 None,
 None,
 None,
 None,
 None]

In [33]:
del d[1]
d._data

[None,
 Bucket(key=1, value=1, is_valid=False),
 None,
 None,
 None,
 None,
 None,
 None]

In [34]:
d[9] = "abc"
d._data

[None,
 Bucket(key=9, value='abc', is_valid=True),
 None,
 None,
 None,
 None,
 None,
 None]

We've now implemented the three things we used to define a dictionary. But there's one thing left we need to modify. Now that we can delete keys we should also resize the dictionary such that the dictionary _shrinks_ if the load factor falls below some threshold value. That way we aren't using a very large dictionary to store a small number of items.

In [35]:
class Dictionary:

    def __init__(self, init_size=8):
        self._items = 0
        self._size = init_size
        self._data: list[Optional[Bucket]] = [None] * self._size
        self._reshape_threshold_grow = 2/3
        self._reshape_threshold_shrink = 1/3

    @property
    def load_factor(self):
        return self._items / self._size
        
        
    def _probes(self, hash_value):
        mask = self._size - 1 # used to take modulus fast
        perturb = hash_value # used to perturb the probe sequence
        probe = hash_value & mask
    
        while True:
            yield probe
    
            perturb >>= 5
            probe = (probe * 5 + perturb + 1) & mask
        

    def _reshape(self, grow: bool):
        if grow:
            self._size <<= 1
        else:
            self._size >>= 1

        new_buckets = [None] * self._size
        for bucket in self._data:
            if bucket and bucket.is_valid:
                h = hash(bucket.key)
                for probe in self._probes(h):
                    if new_buckets[probe] is None:
                        new_buckets[probe] = bucket
                        break
        self._data = new_buckets
        
    def __setitem__(self, key, value):
        h = hash(key)
        new_bucket = Bucket(key, value)
        
        for probe in self._probes(h):
            bucket = self._data[probe]
            if bucket is None or not bucket.is_valid:
                self._data[probe] = new_bucket
                self._items += 1
                break
            elif bucket.key == new_bucket.key:
                self._data[probe] = new_bucket
                break

        if self.load_factor > self._reshape_threshold_grow:
            self._reshape(grow=True)
        elif self.load_factor < self._reshape_threshold_shrink:
            self._reshape(grow=False)
            
    
    def __getitem__(self, item):
        h = hash(item)
        for probe in self._probes(h):
            bucket = self._data[probe]
            if bucket is None:
                raise KeyError(item)
            elif bucket and bucket.key == item:
                return self._data[probe].value

    def __delitem__(self, key):
        h = hash(key)
        for probe in self._probes(h):
            bucket = self._data[probe]
            if bucket is not None and bucket.is_valid and bucket.key == key:
                bucket.is_valid = False
                self._items -= 1
                return
        raise KeyError(key)

In [36]:
d = Dictionary()

for i in range(5):
    d[i] = i
d._data

[Bucket(key=0, value=0, is_valid=True),
 Bucket(key=1, value=1, is_valid=True),
 Bucket(key=2, value=2, is_valid=True),
 Bucket(key=3, value=3, is_valid=True),
 Bucket(key=4, value=4, is_valid=True),
 None,
 None,
 None]

In [37]:
d[5] = 5
d._data

[Bucket(key=0, value=0, is_valid=True),
 Bucket(key=1, value=1, is_valid=True),
 Bucket(key=2, value=2, is_valid=True),
 Bucket(key=3, value=3, is_valid=True),
 Bucket(key=4, value=4, is_valid=True),
 Bucket(key=5, value=5, is_valid=True),
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [38]:
del d[1]
del d[2]
d._data

[Bucket(key=0, value=0, is_valid=True),
 Bucket(key=1, value=1, is_valid=False),
 Bucket(key=2, value=2, is_valid=False),
 Bucket(key=3, value=3, is_valid=True),
 Bucket(key=4, value=4, is_valid=True),
 Bucket(key=5, value=5, is_valid=True),
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [39]:
d[7] = 15
d._data

[Bucket(key=0, value=0, is_valid=True),
 None,
 None,
 Bucket(key=3, value=3, is_valid=True),
 Bucket(key=4, value=4, is_valid=True),
 Bucket(key=5, value=5, is_valid=True),
 None,
 Bucket(key=7, value=15, is_valid=True)]

# How Python does it differently: Dense Tables

So far we've explored how a hashtable is _usually_ implemented. We've implemented our hashtable as a list of ``Bucket`` objects.  Since Python 3.6 the layout of a hashtable has changed. A dictionary like the following:

In [40]:
d = {"one": 1, "two": 2, "three": 3}

Would be represented roughly like this:

In [41]:
hash_table = [
    ('--', '--', '--'),
    (542403711206072985, 'two', 2),
    ('--', '--', '--'),
    (4677866115915370763, 'three', 3),
    ('--', '--', '--'),
    (-1182584047114089363, 'one', 1),
    ('--', '--', '--'),
    ('--', '--', '--')
]

However this isn't optimal because of how Python stores objects, instead Python uses the following layout:

In [42]:
hash_table = [None, 1, None, 2, None, 0, None, None]
entries = [
    (-1182584047114089363, 'one', 1),
    (542403711206072985, 'two', 2),
    (4677866115915370763, 'three', 3),
    ('--', '--', '--'),
    ('--', '--', '--'),
]

In this layout the hashtable only stores indexes to an entries array. Each index takes 1, 2, 4 or 8 bytes depending on the size of the dictionary. Either way, that much less than the 24 bytes taken by our previous Python object! This means that we save a lot of space in the hashtable itself.

# More Theory

## Designing a _GOOD_ hash function

Up to this point we've simply used Python's inbuilt function ``hash`` to compute the hashes for our hash table. But how do we know if this is a _good_ hash function? What does it even mean for a hash function to be good? Let's try to answer some of these questions!

### Some properties we might want in a good hash function

* Uniform Distribution
* One-Way Functions
* Fast

It turns out that designing a function with all of these properties is hard. We'll have to make compromises somewhere. They skill becomes knowing which compromises are right for your application.

### Randomly Generated Hash functions

In theory randomly generated hash functions meet all our requirements but are rarely, if ever used in practice. Why is this? It's because the use too much memory. As the generated hash is random, the only way to get the same hash is to store every hash in memory. This uses to much memory to be used in practice.

### Non-cryptographic hash functions

There are many, many, many possible non-cryptographic hash functions design for hashtables. One of them is FVN-1a. This function:

1. combines the byte with the current hash value (xor)
2. mixes the current hash value (multiplication)


In [ ]:
OFFSET_BASIS = 2166136261
FNV_PRIME = 16777619
HASH_SIZE = 2 ** 32

def fvn1a(data: bytes) -> int:
    h = OFFSET_BASIS
    for byte in data:
        h = h ^ byte
        h = (h * FNV_PRIME) % HASH_SIZE
    return h

Non-cryptographic hash functions perform _really_ well when acting on "normal" data, the drawback is that it's possible to generate adversarial inputs to the function.

### Universal hashing